In [8]:
pip install mysql-connector-python pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import mysql.connector  
import pandas as pd  

# List of CSV file paths
csv_file_paths = [
    'movies_adventure_data.csv',
    'movies_animation_data.csv',
    'movies_crime_data.csv',
    'movies_fantasy_data.csv',
    'movies_biography_data.csv'
]

# Read and merge CSV data
try:
    df_list = [pd.read_csv(file).fillna(0) for file in csv_file_paths]
    final_df = pd.concat(df_list, ignore_index=True)
    final_df[['Duration', 'Rating', 'Vote Count']] = final_df[['Duration', 'Rating', 'Vote Count']].apply(pd.to_numeric, errors='coerce').fillna(0)
    print("CSV files successfully loaded and merged.")
except Exception as e:
    print(f"Error loading CSV files: {e}")
    exit()

# Connect to MySQL
db_config = {'host': 'localhost', 'user': 'root', 'password': 'root', 'database': 'IMBD_Movies_project1'}

# Initialize variables
connection = None
cursor = None  

try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
    print("Connected to MySQL database.")
    
    cursor.execute("CREATE DATABASE IF NOT EXISTS IMBD_Movies_project1")
    # Ensure table exists
    cursor.execute('''CREATE TABLE IF NOT EXISTS movie (
                        id INT AUTO_INCREMENT PRIMARY KEY,
                        Name VARCHAR(255),
                        Genre VARCHAR(100),
                        Duration INT,
                        Rating FLOAT,
                        `Vote Count` INT)''')
    
    # Insert data
    insert_query = """
        INSERT INTO movie (Name, Genre, Duration, Rating, `Vote Count`)
        VALUES (%s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_query, final_df[['Name', 'Genre', 'Duration', 'Rating', 'Vote Count']].values.tolist())
    connection.commit()
    print("Data successfully inserted into MySQL.")

except mysql.connector.Error as err:
    print(f"Database Error: {err}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("MySQL connection closed.")


CSV files successfully loaded and merged.
Connected to MySQL database.
Data successfully inserted into MySQL.
MySQL connection closed.


C:\Users\SUBASH\AppData\Local\Temp\ipykernel_20808\3394019394.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(df_list, ignore_index=True)


In [16]:
import mysql.connector
import pandas as pd

# Connect to MySQL
db_config = {'host': 'localhost', 'user': 'root', 'password': 'root', 'database': 'IMBD_Movies_project1'}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Show Databases
cursor.execute("SHOW DATABASES")
databases = cursor.fetchall()
print("Databases:")
for db in databases:
    print(db[0])



Databases:
company
imbd_movies_project1
information_schema
mysql
performance_schema
supermarketdb
sys
world


In [11]:
if cursor:
    cursor.close()
